### Create the mart layer for the Air Travel warehouse

In [1]:
from google.cloud import bigquery

project_id = "cs378-fa2024"
dataset = "air_travel_mrt"
region = "us-central1"

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset cs378-fa2024.air_travel_mrt


#### Question 1: how many people walk through an airport terminal each hour of the day and therefore how many people can a business, located in an airport, hope to sell to? Join the tsa traffic data with the airport business data

##### Avg number of travelers per hour for different months of the year that a business could hope to sell to

In [27]:
%%bigquery
select b.name as business, a.name as airport, a.city, a.state, ab.terminal,
extract(month from t.event_date) as month_of_year,
round(avg(t.passenger_count), 2) as num_travelers
from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
on ab.business = b.name
join air_travel_int.Airport a
on ab.icao = a.icao
join air_travel_int.TSA_Traffic t
on a.icao = t.airport_icao
group by b.name, a.name, a.city, a.state, ab.terminal, extract(month from t.event_date)
order by month_of_year, num_travelers desc
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,business,airport,city,state,terminal,month_of_year,num_travelers
0,PGA Tour Shop,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
1,Celebrity Bistro & Bar,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
2,Tap Room Bar,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
3,Starbucks,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
4,Deseret News,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
5,CNBC News,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
6,Desert Mart,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
7,California Vintages Wine Bar,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
8,Desert Marketplace,Palm Springs International Airport,Palm Springs,CA,1,1,512.0
9,12th Fairway Bar & Grill,Palm Springs International Airport,Palm Springs,CA,1,1,512.0


##### Same query, but with a more descriptive month field

In [26]:
%%bigquery
select business, airport, city, state, terminal,
case month_of_year
when 1 then 'January'
when 2 then 'February'
when 3 then 'March'
when 4 then 'April'
when 5 then 'May'
when 6 then 'June'
when 7 then 'July'
when 8 then 'August'
when 9 then 'September'
when 10 then 'October'
when 11 then 'November'
when 12 then 'Decemver'
end as month_of_year, num_travelers
from
(
  select b.name as business, a.name as airport, a.city, a.state, ab.terminal, extract(month from t.event_date) as month_of_year,
  round(avg(t.passenger_count), 2) as num_travelers
  from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
  on ab.business = b.name
  join air_travel_int.Airport a
  on ab.icao = a.icao
  join air_travel_int.TSA_Traffic t
  on a.icao = t.airport_icao
  group by b.name, a.name, a.city, a.state, ab.terminal, extract(month from t.event_date)
  order by month_of_year, num_travelers desc
)
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,business,airport,city,state,terminal,month_of_year,num_travelers
0,12th Fairway Bar & Grill,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
1,Desert Marketplace,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
2,PGA Tour Shop,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
3,California Vintages Wine Bar,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
4,Celebrity Bistro & Bar,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
5,Tap Room Bar,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
6,Starbucks,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
7,Desert Mart,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
8,Deseret News,Palm Springs International Airport,Palm Springs,CA,1,January,512.0
9,CNBC News,Palm Springs International Airport,Palm Springs,CA,1,January,512.0


##### Create the mart table

In [29]:
%%bigquery
create or replace table air_travel_mrt.airport_travelers_by_month_year as (
  select business, airport, city, state, terminal,
    case month_of_year
      when 1 then 'January'
      when 2 then 'February'
      when 3 then 'March'
      when 4 then 'April'
      when 5 then 'May'
      when 6 then 'June'
      when 7 then 'July'
      when 8 then 'August'
      when 9 then 'September'
      when 10 then 'October'
      when 11 then 'November'
      when 12 then 'Decemver'
    end as month_of_year, num_travelers
  from
  (
    select b.name as business, a.name as airport, a.city, a.state, ab.terminal,
      extract(month from t.event_date) as month_of_year,
      sum(t.passenger_count) as num_travelers
    from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
    on ab.business = b.name
    join air_travel_int.Airport a
    on ab.icao = a.icao
    join air_travel_int.TSA_Traffic t
    on a.icao = t.airport_icao
    group by b.name, a.name, a.city, a.state, ab.terminal, extract(month from t.event_date)
    order by month_of_year, num_travelers desc
  )
)

Query is running:   0%|          |

""


##### Looking at the same question from a day of week perspective (instead of month of year)

In [25]:
%%bigquery
select b.name as business, a.name as airport, a.city, a.state, ab.terminal,
extract(dayofweek from t.event_date) as weekday,
round(avg(t.passenger_count), 2) as num_travelers
from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
on ab.business = b.name
join air_travel_int.Airport a
on ab.icao = a.icao
join air_travel_int.TSA_Traffic t
on a.icao = t.airport_icao
group by b.name, a.name, a.city, a.state, ab.terminal, extract(dayofweek from t.event_date)
order by weekday, num_travelers desc
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,business,airport,city,state,terminal,weekday,num_travelers
0,Burger King,John Glenn Columbus International Airport,Columbus,OH,1,1,363.00
1,Charleys Cheesesteaks,John Glenn Columbus International Airport,Columbus,OH,1,1,363.00
2,The Bluegrass Pub,Blue Grass Airport,Lexington KY,KY,1,1,224.00
3,The Club at Bluegrass,Blue Grass Airport,Lexington KY,KY,1,1,224.00
4,Bluegrass Market,Blue Grass Airport,Lexington KY,KY,1,1,224.00
5,The Market,Blue Grass Airport,Lexington KY,KY,1,1,224.00
6,Bluegrass Coffee,Blue Grass Airport,Lexington KY,KY,1,1,224.00
7,Vino,Blue Grass Airport,Lexington KY,KY,1,1,224.00
8,Berry Austin,Austin Bergstrom International Airport,Austin,TX,1,1,167.08
9,Brighton,Austin Bergstrom International Airport,Austin,TX,1,1,167.08


##### Same query, but now with a more descriptive day of week

In [24]:
%%bigquery
select business, airport, city, state, terminal,
case weekday
  when 1 then 'Sunday'
  when 2 then 'Monday'
  when 3 then 'Tuesday'
  when 4 then 'Wednesday'
  when 5 then 'Thursday'
  when 6 then 'Friday'
  when 7 then 'Saturday'
end as weekday, num_travelers
from
  (select b.name as business, a.name as airport, a.city, a.state, ab.terminal, extract(dayofweek from t.event_date) as weekday,
  round(avg(t.passenger_count), 2) as num_travelers
  from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
  on ab.business = b.name
  join air_travel_int.Airport a
  on ab.icao = a.icao
  join air_travel_int.TSA_Traffic t
  on a.icao = t.airport_icao
  group by b.name, a.name, a.city, a.state, ab.terminal, extract(dayofweek from t.event_date)
  order by weekday, num_travelers desc)
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,business,airport,city,state,terminal,weekday,num_travelers
0,Burger King,John Glenn Columbus International Airport,Columbus,OH,1,Sunday,363.00
1,Charleys Cheesesteaks,John Glenn Columbus International Airport,Columbus,OH,1,Sunday,363.00
2,Bluegrass Market,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
3,Vino,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
4,The Market,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
5,The Bluegrass Pub,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
6,The Club at Bluegrass,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
7,Bluegrass Coffee,Blue Grass Airport,Lexington KY,KY,1,Sunday,224.00
8,Brighton,Austin Bergstrom International Airport,Austin,TX,1,Sunday,167.08
9,Berry Austin,Austin Bergstrom International Airport,Austin,TX,1,Sunday,167.08


##### Create the mart table

In [28]:
%%bigquery
create or replace table air_travel_mrt.airport_travelers_by_weekday as (
  select business, airport, city, state, terminal,
  case weekday
    when 1 then 'Sunday'
    when 2 then 'Monday'
    when 3 then 'Tuesday'
    when 4 then 'Wednesday'
    when 5 then 'Thursday'
    when 6 then 'Friday'
    when 7 then 'Saturday'
  end as weekday, num_travelers
  from
    (select b.name as business, a.name as airport, a.city, a.state, ab.terminal, extract(dayofweek from t.event_date) as weekday,
    round(avg(t.passenger_count), 2) as num_travelers
    from air_travel_int.Airport_Businesses ab join air_travel_int.Business b
    on ab.business = b.name
    join air_travel_int.Airport a
    on ab.icao = a.icao
    join air_travel_int.TSA_Traffic t
    on a.icao = t.airport_icao
    group by b.name, a.name, a.city, a.state, ab.terminal, extract(dayofweek from t.event_date)
    order by weekday, num_travelers desc)
)

Query is running:   0%|          |

""


#### Question 2: What are the most common and least common food items available for purchase at different US airports?

In [38]:
%%bigquery
select a.name as airport, menu_item, count(*) as count
from air_travel_int.Menu_Items m join air_travel_int.Business b
on m.business_name = b.name
join air_travel_int.Airport_Businesses ab on b.name = ab.business
join air_travel_int.Airport a on ab.icao = a.icao
group by a.name, menu_item
order by a.name, count(*) desc
limit 20

Query is running:   0%|          |

Downloading:   0%|          |

,airport,menu_item,count
0,Akron Canton Regional Airport,Great Lakes Eliot Ness Amber Lager,1
1,Akron Canton Regional Airport,Great Lakes Dortmunder Gold Lager,1
2,Akron Canton Regional Airport,Breakfast Burrito,1
3,Akron Canton Regional Airport,Pasta Salad,1
4,Akron Canton Regional Airport,Great Lakes Edmund Fitzgerald Porter,1
5,Akron Canton Regional Airport,Breakfast Sandwich,1
6,Akron Canton Regional Airport,Fruit Salad,1
7,Akron Canton Regional Airport,Chicken Salad Sandwich,1
8,Akron Canton Regional Airport,Turkey Wrap,1
9,Albert J Ellis Airport,Burger and Fries,1


##### Create the mart table

In [39]:
%%bigquery
create or replace table air_travel_mrt.food_items_popularity_by_airport as
  select a.name as airport, menu_item, count(*) as count
  from air_travel_int.Menu_Items m join air_travel_int.Business b
  on m.business_name = b.name
  join air_travel_int.Airport_Businesses ab on b.name = ab.business
  join air_travel_int.Airport a on ab.icao = a.icao
  group by a.name, menu_item
  order by a.name, count(*) desc

Query is running:   0%|          |

""


#### Question 3: What are the most common and least common types of businesses present at different US airports?

In [43]:
%%bigquery
select a.name as airport, b.category, count(*) as count
from air_travel_int.Menu_Items m join air_travel_int.Business b
on m.business_name = b.name
join air_travel_int.Airport_Businesses ab on b.name = ab.business
join air_travel_int.Airport a on ab.icao = a.icao
group by a.name, b.category
order by a.name, count(*) desc
limit 20

Query is running:   0%|          |

Downloading:   0%|          |

,airport,category,count
0,Akron Canton Regional Airport,Restaurant,6
1,Akron Canton Regional Airport,Bar/Restaurant,3
2,Albert J Ellis Airport,Dining,6
3,Albuquerque International Sunport,Coffee Shop,9
4,Albuquerque International Sunport,Bar/Restaurant,6
5,Albuquerque International Sunport,Restaurant,6
6,Albuquerque International Sunport,Food & Drink,3
7,Albuquerque International Sunport,Dining,3
8,Appleton International Airport,Dining,6
9,Appleton International Airport,Restaurant,3


##### Create the mart table

In [42]:
%%bigquery
create or replace table air_travel_mrt.business_type_popularity_by_airport as
  select a.name as airport, b.category, count(*) as count
  from air_travel_int.Menu_Items m join air_travel_int.Business b
  on m.business_name = b.name
  join air_travel_int.Airport_Businesses ab on b.name = ab.business
  join air_travel_int.Airport a on ab.icao = a.icao
  group by a.name, b.category
  order by a.name, count(*) desc

Query is running:   0%|          |

""


#### Question 4: What are the best and worst airports in the world and in the US based on the sentiment and number of traveler reviews?

##### Highest-rated airports in the world

In [52]:
%%bigquery
select a.name as airport_name, a.city, a.country, ar.sentiment, count(*) as num_reviews
from air_travel_int.Airport a join air_travel_int.Airport_Review ar
on a.icao = ar.icao
where ar.sentiment = 'positive'
and ar.relevant = true
group by a.name, a.city, a.country, ar.sentiment
order by count(*) desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airport_name,city,country,sentiment,num_reviews
0,Beja Airport / Airbase,Beja (madeira),Portugal,positive,15
1,Billy Bishop Toronto City Centre Airport,Toronto,Canada,positive,11
2,Kawartha Lakes (Lindsay) Airport,Lindsay,Canada,positive,8
3,Birsa Munda Airport,Ranchi,India,positive,7
4,Mansfield Lahm Regional Airport,Mansfield,United States,positive,7
5,Bokaro Airport,Bokaro,India,positive,6
6,Brantford Municipal Airport,Brantford,Canada,positive,6
7,Kingston Norman Rogers Airport,Kingston,Canada,positive,6
8,Ottawa Macdonald-Cartier International Airport,Ottawa,Canada,positive,6
9,Legazpi City International Airport,Legazpi,Philippines,positive,6


##### Create the mart table

In [57]:
%%bigquery
create or replace table air_travel_mrt.highest_rated_airports_world as
  select a.name as airport_name, a.city, a.country, ar.sentiment, count(*) as num_reviews
  from air_travel_int.Airport a join air_travel_int.Airport_Review ar
  on a.icao = ar.icao
  where ar.sentiment = 'positive'
  and ar.relevant = true
  group by a.name, a.city, a.country, ar.sentiment
  order by count(*) desc

Query is running:   0%|          |

""


##### Lowest-rated airports in the world

In [53]:
%%bigquery
select a.name as airport_name, a.city, a.country, ar.sentiment, count(*) as num_reviews
from air_travel_int.Airport a join air_travel_int.Airport_Review ar
on a.icao = ar.icao
where ar.sentiment = 'negative'
and ar.relevant = true
group by a.name, a.city, a.country, ar.sentiment
order by count(*) desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airport_name,city,country,sentiment,num_reviews
0,Douala International Airport,Douala,Cameroon,negative,227
1,Diosdado Macapagal International Airport,Angeles City,Philippines,negative,21
2,Beja Airport / Airbase,Beja (madeira),Portugal,negative,9
3,OR Tambo International Airport,Johannesburg,South Africa,negative,7
4,Gaya Airport,Gaya,India,negative,6
5,Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,negative,6
6,Essex County Airport,Caldwell,United States,negative,5
7,Sydney Bankstown Airport,Sydney,Australia,negative,4
8,Kukës Airport,Kukës,Albania,negative,4
9,Sudbury Airport,Sudbury,Canada,negative,3


##### Create the mart table

In [58]:
%%bigquery
create or replace table air_travel_mrt.lowest_rated_airports_world as
  select a.name as airport_name, a.city, a.country, ar.sentiment, count(*) as num_reviews
  from air_travel_int.Airport a join air_travel_int.Airport_Review ar
  on a.icao = ar.icao
  where ar.sentiment = 'negative'
  and ar.relevant = true
  group by a.name, a.city, a.country, ar.sentiment
  order by count(*) desc

Query is running:   0%|          |

""


##### Highest-rated airports in the US

In [59]:
%%bigquery
select a.name as airport_name, a.city, a.state, ar.sentiment, count(*) as num_reviews
from air_travel_int.Airport a join air_travel_int.Airport_Review ar
on a.icao = ar.icao
where ar.sentiment = 'positive'
and ar.relevant = true
and a.country = 'United States'
group by a.name, a.city, a.state, ar.sentiment
order by count(*) desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airport_name,city,state,sentiment,num_reviews
0,Mansfield Lahm Regional Airport,Mansfield,None,positive,7
1,Burlington International Airport,Burlington,VT,positive,5
2,Teterboro Airport,Teterboro,None,positive,4
3,Hartsfield Jackson Atlanta International Airport,Atlanta,None,positive,4
4,Republic Airport,Farmingdale,None,positive,4
5,Capital City Airport,Harrisburg,None,positive,3
6,Detroit Metropolitan Wayne County Airport,Detroit,MI,positive,3
7,Reid-Hillview Airport of Santa Clara County,San Jose,None,positive,3
8,Tri-County Regional Airport,Lone Rock,None,positive,3
9,Portland International Airport,Portland,OR,positive,3


##### Create the mart table

In [60]:
%%bigquery
create or replace table air_travel_mrt.highest_rated_airports_us as
  select a.name as airport_name, a.city, a.state, ar.sentiment, count(*) as num_reviews
  from air_travel_int.Airport a join air_travel_int.Airport_Review ar
  on a.icao = ar.icao
  where ar.sentiment = 'positive'
  and ar.relevant = true
  and a.country = 'United States'
  group by a.name, a.city, a.state, ar.sentiment
  order by count(*) desc

Query is running:   0%|          |

""


##### Lowest-rated airports in the US

In [61]:
%%bigquery
select a.name as airport_name, a.city, a.state, ar.sentiment, count(*) as num_reviews
from air_travel_int.Airport a join air_travel_int.Airport_Review ar
on a.icao = ar.icao
where ar.sentiment = 'negative'
and ar.relevant = true
and a.country = 'United States'
group by a.name, a.city, a.state, ar.sentiment
order by count(*) desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airport_name,city,state,sentiment,num_reviews
0,Essex County Airport,Caldwell,None,negative,5
1,Hazleton Municipal Airport,Hazleton,None,negative,3
2,Laurence G Hanscom Field,Bedford,None,negative,3
3,Los Angeles International Airport,Los Angeles,CA,negative,3
4,La Guardia Airport,New York,NY,negative,2
5,Capital City Airport,Harrisburg,None,negative,2
6,Manassas Regional Airport/Harry P. Davis Field,Manassas,None,negative,2
7,Reid-Hillview Airport of Santa Clara County,San Jose,None,negative,2
8,Chicago Meigs Airport,Chicago,None,negative,2
9,Miami International Airport,Miami,FL,negative,2


##### Create the mart table

In [62]:
%%bigquery
create or replace table air_travel_mrt.lowest_rated_airports_us as
  select a.name as airport_name, a.city, a.state, ar.sentiment, count(*) as num_reviews
  from air_travel_int.Airport a join air_travel_int.Airport_Review ar
  on a.icao = ar.icao
  where ar.sentiment = 'negative'
  and ar.relevant = true
  and a.country = 'United States'
  group by a.name, a.city, a.state, ar.sentiment
  order by count(*) desc

Query is running:   0%|          |

""


#### Question 5: Which US airlines are associated with the least and most flight delays? Note: this data is only available for US flights.

##### First, look at this question from the perspective of number of minutes delayed (as measured by arrival time at destination airport)

In [67]:
%%bigquery
select al.name as airline, ap.name as airport, ap.city, ap.state, sum(fd.arr_delay_min) as delay_minutes
from air_travel_int.Airport ap join air_travel_int.Flight_Delays fd
on ap.icao = fd.airport_icao
join air_travel_int.Airline al
on fd.airline_id = al.id
where fd.arr_delay_min is not null
group by al.name, ap.name, ap.city, ap.state
order by delay_minutes desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airline,airport,city,state,delay_minutes
0,Delta Air Lines,Hartsfield Jackson Atlanta International Airport,Atlanta,None,40109644
1,American Airlines,Dallas Fort Worth International Airport,Dallas-Fort Worth,TX,38594115
2,Atlantic Southeast Airlines,Hartsfield Jackson Atlanta International Airport,Atlanta,None,21461111
3,United Airlines,Chicago O'Hare International Airport,Chicago,IL,20832721
4,American Eagle Airlines,Chicago O'Hare International Airport,Chicago,IL,19953115
5,American Airlines,Chicago O'Hare International Airport,Chicago,IL,19467816
6,SkyWest,Chicago O'Hare International Airport,Chicago,IL,15788771
7,American Eagle Airlines,Dallas Fort Worth International Airport,Dallas-Fort Worth,TX,15663577
8,JetBlue Airways,John F Kennedy International Airport,New York,NY,15511382
9,SkyWest,San Francisco International Airport,San Francisco,CA,14906245


##### Create the mart table

In [68]:
%%bigquery
create or replace table air_travel_mrt.flight_delay_length_by_airline as
  select al.name as airline, ap.name as airport, ap.city, ap.state, sum(fd.arr_delay_min) as delay_minutes
  from air_travel_int.Airport ap join air_travel_int.Flight_Delays fd
  on ap.icao = fd.airport_icao
  join air_travel_int.Airline al
  on fd.airline_id = al.id
  where fd.arr_delay_min is not null
  group by al.name, ap.name, ap.city, ap.state
  order by delay_minutes desc

Query is running:   0%|          |

""


##### Second, look at this question from the perspective of frequency of delay by year

In [74]:
%%bigquery
select airline, airport, city, state, round(avg(delay_frequency), 2) as delay_frequency
from
  (select al.name as airline, ap.name as airport, ap.city, ap.state, extract(year from fd.event_month) as year, count(fd.arr_delay_min) as delay_frequency
  from air_travel_int.Airport ap join air_travel_int.Flight_Delays fd
  on ap.icao = fd.airport_icao
  join air_travel_int.Airline al
  on fd.airline_id = al.id
  where fd.arr_delay_min > 0
  group by al.name, ap.name, ap.city, ap.state, extract(year from fd.event_month)
  order by delay_frequency desc)
group by airline, airport, city, state
order by delay_frequency desc
limit 12

Query is running:   0%|          |

Downloading:   0%|          |

,airline,airport,city,state,delay_frequency
0,Trans States Airlines,Pittsburgh International Airport,Pittsburgh,PA,12.00
1,Pinnacle Airlines,Stewart International Airport,Newburgh,NY,12.00
2,Trans States Airlines,Bangor International Airport,Bangor,ME,12.00
3,Mesa Airlines,MBS International Airport,Saginaw,MI,12.00
4,Trans States Airlines,Ronald Reagan Washington National Airport,Washington,VA,12.00
5,GoJet Airlines,Norman Y. Mineta San Jose International Airport,San Jose,CA,12.00
6,Trans States Airlines,Westchester County Airport,White Plains,NY,12.00
7,Pinnacle Airlines,Chippewa County International Airport,Sault Ste Marie,MI,12.00
8,Pinnacle Airlines,Delta County Airport,Escanaba,MI,12.00
9,SkyWest,Barkley Regional Airport,PADUCAH,None,11.85


##### Create the mart table

In [75]:
%%bigquery
create or replace table air_travel_mrt.flight_delay_frequency_by_airline as
  select airline, airport, city, state, round(avg(delay_frequency), 2) as delay_frequency
  from
    (select al.name as airline, ap.name as airport, ap.city, ap.state, extract(year from fd.event_month) as year, count(fd.arr_delay_min) as delay_frequency
    from air_travel_int.Airport ap join air_travel_int.Flight_Delays fd
    on ap.icao = fd.airport_icao
    join air_travel_int.Airline al
    on fd.airline_id = al.id
    where fd.arr_delay_min > 0
    group by al.name, ap.name, ap.city, ap.state, extract(year from fd.event_month)
    order by delay_frequency desc)
  group by airline, airport, city, state
  order by delay_frequency desc

Query is running:   0%|          |

""
